# 크롤러 환경설정

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd
import json
import os, sys
import tweepy

sys.path.append(os.pardir)
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
key_path = 'C:/Users/dkdud/Untitled Folder/api_key.json'
with open(key_path,'r') as f:
    key = json.load(f)

# 개인정보 인증을 요청하는 Handler입니다.
auth = tweepy.OAuthHandler(key['CONSUMER_KEY'], key['CONSUMER_SECRET'])

# 인증 요청을 수행합니다.
auth.set_access_token(key['ACCESS_TOKEN_KEY'], key['ACCESS_TOKEN_SECRET'])

# twitter API를 사용하기 위한 준비입니다.
api = tweepy.API(auth,wait_on_rate_limit=True)

In [25]:
#인스타 게시물을 불러오기 위한 사전작업 단계
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='data.campuss'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'data.2021'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)

def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

    
def get_content(driver):
    data=[]
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    tag_list =[]
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    tag_data = ''.join(tag_data)
    for tag_one in tag_data:
        tag_list.append(tag_one)
    print(content)
    print(type(content))
    print(tag_data)
    print(type(tag_data))
    data.append(content+tag_data)
    return data

TimeoutException: Message: 


In [23]:
def insta_crawling(driver,keyword, target):
    
    url = insta_searching(keyword)
    driver.implicitly_wait(7)
    driver.get(url)
    time.sleep(7)

    click_first(driver)

    for i in range(target):
        try:
            get_content(driver)
            print('a')
            
            next_page(driver)
        except:
            print('b')
            time.sleep(9)
            next_page(driver)
    df=pd.DataFrame(data)
    df.columns=['본문']
    df['본문'] = df['본문'].apply(lambda x : clean_text(x))
    df = df.drop_duplicates(subset=['본문'])
    return df

In [7]:
#트위터 게시물 가져오는 단계
def twitter_crawling(keyword):
    columns = ['본문']
    df = pd.DataFrame(columns=columns)

    max_tweets = 1000
    searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]

    for tweet in searched_tweets:
        tweet_json = tweet._json
        tweet_text = tweet_json['text']
        row = [tweet_text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)

    df['본문'] = df['본문'].apply(lambda x : clean_text(x))
    df = df.drop_duplicates(subset=['본문'])
    
    return df


In [11]:
keyword = '풀무원'
target = 5

In [26]:
insta_df = insta_crawling(driver,keyword, target)

b
b
b
b
b


NameError: name 'data' is not defined

In [ ]:
twitter_df = twitter_crawling(keyword)

In [ ]:
pd.concat([twitter_df, insta_df],join='outer')

In [ ]:
df_list = [insta_df, twitter_df]

In [ ]:
sns_df = pd.DataFrame(df_list)